# Trabalho 1 - Coleta, Preparação e Análise de Dados

## Webscrapping em ambiente real

### Objetivos da tarefa:

- Faça scraping para obter os 250 filmes com as maiores avaliações do IMDB. Devem
  ser obtidos: Título, Duração, url do poster e nota imdb.

- Faça scraping das páginas específicas dos 250 filmes obtidos no item anterior.
  Obtenha dessa página a popularidade e a listagem do elenco principal (incluindo nome
  do ator/atriz e da personagem).

- Salve as informações obtidas em arquivo json


In [158]:
# imports
import re
import json
from bs4 import BeautifulSoup
from dataclasses import field
from dataclasses import dataclass

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [159]:
# criando o driver e o site do imdb
driver = webdriver.Chrome()
url = 'https://www.imdb.com'

driver.get(url)

In [160]:
# clicando no menu lateral
menu = driver.find_element(By.XPATH, r"//*[@id='imdbHeader-navDrawerOpen']")
menu.click()

In [161]:
# entrando na página dos 250 melhores filmes
top_250 = driver.find_element(By.LINK_TEXT, r'Top 250 Movies')
url_top250 = top_250.get_attribute("href")
driver.get(url_top250)

In [162]:
# pega as tags de cada filme na página
tag_ul = driver.find_element(By.XPATH, r'//*[@id="__next"]/main/div/div[3]/section/div/div[2]/div/ul')
movies_list = tag_ul.find_elements(By.TAG_NAME, "li")
    
# exemplo de como é o texto de cada variável
print(movies_list[0].text)

1. The Shawshank Redemption
1994
2h 22m
R
9.3
 (2.9M)
Rate


In [163]:
# classe que monta a estrutura de dados para armazenar um filme
@dataclass
class Movie:
    title: str
    year: int
    duration: int #duração em minutos
    poster_url: str
    classification: str
    score: float
    popularity: float
    cast: list[str] = field(default_factory=list)

pages = []
# método que passa por cada tag de filme na página da lista e cria um objeto Movie
def cria_filme(movie_tag):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    title = movie_tag.find_element(By.CLASS_NAME, "ipc-title__text").text # pegando o texto de classe "title"
    title = re.match(r'^\d+\. (.*)', title).group(1) # removendo o índice antes do nome do filme
    
    year = int(movie_tag.find_element(By.XPATH, r'./div[2]/div/div/div[2]/span[1]').text) # procura o ano do filme pelo caminho
    
    # procurando o texto da duração do filme e converte para minutos
    time = re.match(r'(\d+)h (\d+)m', movie_tag.text.split('\n')[2])
    duration = int(time.group(1)) * 60 + int(time.group(2)) if time else None
    
    # usando o soup, encontrando a nota do filme
    score_tag = soup.find('span', {'data-testid' : 'ratingGroup--imdb-rating'})
    score = score_tag.find('span', class_='ipc-rating-star--rating').text
    
    url = movie_tag.find_element(By.XPATH, ".//div[2]/img").get_attribute("src") # guarda o link do poster do filme
    
    # encontrando a string de classificação do filme, e se não encontra, atribui o valor "indisponível"
    classification_tag = movie_tag.find_element(By.CLASS_NAME, "sc-b189961a-7")
    try:
        classification_tag = classification_tag.find_element(By.XPATH, ".//span[contains(@class, 'cli-title-metadata-item')][3]")
    except NoSuchElementException:
        classification_tag = None
    classification = classification_tag.text if classification_tag else "Unavailable"
    
    # guarda o link da página do filme numa variável global
    link = movie_tag.find_element(By.CLASS_NAME, "ipc-title-link-wrapper").get_attribute("href")
    pages.append(link)
        
    return Movie(title, year, duration, url, classification, score, None, None)

In [164]:
# segunda parte da tarefa: iterando sobre a página de cada filme
def movie_page_info(url):
    driver.get(url) # vai para a página do filme e cria um objeto soup
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    
    popularity_tag = soup.find('div', {'data-testid': 'hero-rating-bar__popularity__score'}) # encontra o número de popularidade
    popularity = popularity_tag.text if popularity_tag else None
    
    # guarda os atores principais do filme em uma lista
    cast_tag = soup.find('section', {'data-testid': 'title-cast'})
    actors = []
    actor_tags = cast_tag.find_all('a', {'data-testid': 'title-cast-item__actor'})
    for tag in actor_tags:
        actors.append(tag.text)
        
    return popularity, actors

In [165]:
movies = []
# cria uma lista com os objetos Movie
for movie_tag in movies_list:
    movie = cria_filme(movie_tag)
    movies.append(movie)

# da lista, adiciona os atributos adicionais aos objetos
for i, movie in enumerate(movies):
    movie.popularity, movie.cast = movie_page_info(pages[i])

In [166]:
# converte os items da lista em um dicionário e, por fim, em um arquivo.json
dictionary = []
for movie in movies:
    dictionary.append(movie.__dict__)
with open('top250_movies.json', 'w') as file:
    json.dump(dictionary, file,ensure_ascii=False, indent=4)